# Multi Axes Figures 

In [ ]:
%matplotlib widget

import matplotlib.pyplot as plt
import numpy as np


# Helper function used for visualization in the following examples
def identify_axes(ax_dict, fontsize=48):
    """
    Helper to identify the Axes in the examples below.

    Draws the label in a large font in the center of the Axes.
    Works with the returns of subplot_mosaic or subplots.

    Parameters
    ----------
    ax_dict : dict[str, Axes] or ndarray[Axes]
        Mapping between the title / label and the Axes.
        If ndarray, labels the indices
    fontsize : int, optional
        How big the label should be.
    """
    if isinstance(ax_dict, np.ndarray):
        ax_dict = {repr(idx): ax for idx, ax in np.ndenumerate(ax_dict)}
    kw = dict(ha="center", va="center", fontsize=fontsize, color="darkgrey")
    for k, ax in ax_dict.items():
        ax.text(0.5, 0.5, k, transform=ax.transAxes, **kw)

# Some example data to display
x = np.linspace(0, 2 * np.pi, 400)
y = np.sin(x ** 2)

## subplots

### Stacking subplots

The first two optional arguments of `.pyplot.subplots` define the number of
rows and columns of the subplot grid.

When stacking in one direction only, the returned ``axs`` is a 1D numpy array
containing the list of created Axes.

In [ ]:
fig, axs = plt.subplots(2)
fig.suptitle('Vertically stacked subplots')
axs[0].plot(x, y)
axs[1].plot(x, -y)

If you are creating just a few Axes, it's handy to unpack them immediately to
dedicated variables for each Axes. That way, we can use ``ax1`` instead of
the more verbose ``axs[0]``.



In [ ]:
fig, (ax1, ax2) = plt.subplots(2)
fig.suptitle('Vertically stacked subplots')
ax1.plot(x, y)
ax2.plot(x, -y)

To obtain side-by-side subplots, pass parameters ``1, 2`` for one row and two
columns.



In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
fig.suptitle('Horizontally stacked subplots')
ax1.plot(x, y)
ax2.plot(x, -y)

### Grid of subplots

If we want a 2x2 grid we can use `Figure.subplots` which returns a 2D array
of `Axes` which we can index into to do our plotting.



In [ ]:
np.random.seed(19680801)
hist_data = np.random.randn(1_500)


fig = plt.figure(layout="constrained")
# Note: "constrained" is used to make the figure use a particular layout mangager
# this helps make sure things like tick labels do not overlap
# We will cover this and other similar tools in section 5

ax_array = fig.subplots(2, 2, squeeze=False)

ax_array[0, 0].bar(["a", "b", "c"], [5, 7, 9])
ax_array[0, 1].plot([1, 2, 3])
ax_array[1, 0].hist(hist_data, bins="auto")
ax_array[1, 1].imshow([[1, 2], [2, 1]])

identify_axes(ax_array)

### Sharing axes

By default, each Axes is scaled individually. Thus, if the ranges are
different the tick values of the subplots do not align.

Hint: Use the pan/zoom capabilities to get a more thourough understanding of how shared axes behave.

In [ ]:
fig, (ax1, ax2) = plt.subplots(2)
fig.suptitle('Axes values are scaled individually by default')
ax1.plot(x, y)
ax2.plot(x + 1, -y)

You can use *sharex* or *sharey* to align the horizontal or vertical axis.



In [ ]:
fig, (ax1, ax2) = plt.subplots(2, sharex=True)
fig.suptitle('Aligning x-axis using sharex')
ax1.plot(x, y)
ax2.plot(x + 1, -y)

Setting *sharex* or *sharey* to ``True`` enables global sharing across the
whole grid, i.e. also the y-axes of vertically stacked subplots have the
same scale when using ``sharey=True``.



In [ ]:
fig, axs = plt.subplots(3, sharex=True, sharey=True)
fig.suptitle('Sharing both axes')
axs[0].plot(x, y ** 2)
axs[1].plot(x, 0.3 * y, 'o')

axs[2].plot(x, y, '+')

For subplots that are sharing axes one set of tick labels is enough. Tick
labels of inner Axes are automatically removed by *sharex* and *sharey*.
Still there remains an unused empty space between the subplots.

To precisely control the positioning of the subplots, one can explicitly
create a `GridSpec` with `Figure.add_gridspec`, and then call its
`GridSpecBase.subplots` method.  For example, we can reduce the height
between vertical subplots using ``add_gridspec(hspace=0)``.

`label_outer` is a handy method to remove labels and ticks from subplots
that are not at the edge of the grid.



In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(3, hspace=0)
axs = gs.subplots(sharex=True, sharey=True)
fig.suptitle('Sharing both axes')
axs[0].plot(x, y ** 2)
axs[1].plot(x, 0.3 * y, 'o')
axs[2].plot(x, y, '+')

# Hide x labels and tick labels for all but bottom plot.
for ax in axs:
    ax.label_outer()

Apart from ``True`` and ``False``, both *sharex* and *sharey* accept the
values 'row' and 'col' to share the values only per row or column.



In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(2, 2, hspace=0, wspace=0)
(ax1, ax2), (ax3, ax4) = gs.subplots(sharex='col', sharey='row')
fig.suptitle('Sharing x per column, y per row')
ax1.plot(x, y)
ax2.plot(x, y**2, 'tab:orange')
ax3.plot(x + 1, -y, 'tab:green')
ax4.plot(x + 2, -y**2, 'tab:red')

for ax in fig.get_axes():
    ax.label_outer()

If you want a more complex sharing structure, you can first create the
grid of axes with no sharing, and then call `Axes.sharex` or
`Axes.sharey` to add sharing info a posteriori.



In [ ]:
fig, axs = plt.subplots(2, 2)
axs[0, 0].plot(x, y)
axs[0, 0].set_title("main")
axs[1, 0].plot(x, y**2)
axs[1, 0].set_title("shares x with main")
axs[1, 0].sharex(axs[0, 0])
axs[0, 1].plot(x + 1, y + 1)
axs[0, 1].set_title("unrelated")
axs[1, 1].plot(x + 2, y + 2)
axs[1, 1].set_title("also unrelated")
fig.tight_layout()

### Polar axes

The parameter *subplot_kw* of `pyplot.subplots` controls the subplot
properties (see also `Figure.add_subplot`). In particular, this can be used
to create a grid of polar Axes.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, subplot_kw=dict(projection='polar'))
ax1.plot(x, y)
ax2.plot(x, y ** 2)

plt.show()

## Subplot mosaic


Laying out Axes in a Figure in a non-uniform grid can be both tedious
and verbose.  For dense, even grids we have `subplots` but for
more complex layouts, such as Axes that span multiple columns / rows
of the layout or leave some areas of the Figure blank, you can use
`GridSpec` (We will explore this in more depth in the next section) or
manually place your axes.  `subplot_mosaic` aims to provide an
interface to visually lay out your axes (as either ASCII art or nested
lists) to streamline this process.

This interface naturally supports naming your axes.
`subplot_mosaic` returns a dictionary keyed on the
labels used to lay out the Figure.  By returning data structures with
names, it is easier to write plotting code that is independent of the
Figure layout.


Using `subplot_mosaic` we can produce the same figure as the 2x2 grid example above but give the
axes semantic names



In [ ]:
fig = plt.figure(layout="constrained")
ax_dict = fig.subplot_mosaic(
    [
        ["bar", "plot"],
        ["hist", "image"],
    ],
)
ax_dict["bar"].bar(["a", "b", "c"], [5, 7, 9])
ax_dict["plot"].plot([1, 2, 3])
ax_dict["hist"].hist(hist_data)
ax_dict["image"].imshow([[1, 2], [2, 1]])
identify_axes(ax_dict)

A key difference between `subplots` and
`subplot_mosaic` is the return value. While the former
returns an array for index access, the latter returns a dictionary
mapping the labels to the `Axes` instances created



In [ ]:
print(ax_dict)

#### Exercise: Make an alignment chart


### String short-hand

By restricting our axes labels to single characters we can
"draw" the Axes we want as "ASCII art".  The following



In [ ]:
mosaic = """
    AB
    CD
    """

will give us 4 Axes laid out in a 2x2 grid and generates the same
figure mosaic as above (but now labeled with ``{"A", "B", "C",
"D"}`` rather than ``{"bar", "plot", "hist", "image"}``).



In [ ]:
fig = plt.figure(layout="constrained")
ax_dict = fig.subplot_mosaic(mosaic)
identify_axes(ax_dict)

Alternatively, you can use the more compact string notation



In [ ]:
mosaic = "AB;CD"

will give you the same composition, where the ``";"`` is used
as the row separator instead of newline.



In [ ]:
fig = plt.figure(layout="constrained")
ax_dict = fig.subplot_mosaic(mosaic)
identify_axes(ax_dict)

### Axes spanning multiple rows/columns

Something we can do with `subplot_mosaic`, that we cannot
do with `subplots`, is to specify that an Axes should span
several rows or columns.



If we want to re-arrange our four Axes to have ``"C"`` be a horizontal
span on the bottom and ``"D"`` be a vertical span on the right we would do



In [ ]:
axd = plt.figure(layout="constrained").subplot_mosaic(
    """
    ABD
    CCD
    """
)
identify_axes(axd)

If we do not want to fill in all the spaces in the Figure with Axes,
we can specify some spaces in the grid to be blank



In [ ]:
axd = plt.figure(layout="constrained").subplot_mosaic(
    """
    A.C
    BBB
    .D.
    """
)
identify_axes(axd)

If we prefer to use another character (rather than a period ``"."``)
to mark the empty space, we can use *empty_sentinel* to specify the
character to use.



In [ ]:
axd = plt.figure(layout="constrained").subplot_mosaic(
    """
    aX
    Xb
    """,
    empty_sentinel="X",
)
identify_axes(axd)

Internally there is no meaning attached to the letters we use, any
Unicode code point is valid!



In [ ]:
axd = plt.figure(layout="constrained").subplot_mosaic(
    """αб
       ℝ☢"""
)
identify_axes(axd)

It is not recommended to use white space as either a label or an
empty sentinel with the string shorthand because it may be stripped
while processing the input.

### Controlling width/height of Axes

This feature is built on top of `gridspec` and you can pass the
keyword arguments through to the underlying `GridSpec`
(the same as `subplots`).

In this case we want to use the input to specify the arrangement,
but set the relative widths of the rows / columns.  For convenience,
`GridSpec`'s *height_ratios* and *width_ratios* are exposed in the
`subplot_mosaic` calling sequence.

In [ ]:
axd = plt.figure(layout="constrained").subplot_mosaic(
    """
    .a.
    bAc
    .d.
    """,
    # set the height ratios between the rows
    height_ratios=[1, 3.5, 1],
    # set the width ratios between the columns
    width_ratios=[1, 3.5, 1],
)
identify_axes(axd)

### Subplot arguments

We can also pass through arguments used to create the subplots
(again, the same as `subplots`) which will apply to all
of the Axes created.



In [ ]:
axd = plt.figure(layout="constrained").subplot_mosaic(
    "AB", subplot_kw={"projection": "polar"}
)
identify_axes(axd)

### Per-Axes subplot keyword arguments

If you need to control the parameters passed to each subplot individually use
*per_subplot_kw* to pass a mapping between the Axes identifiers (or
tuples of Axes identifiers) to dictionaries of keywords to be passed.

(New in Matplotlib 3.7)




In [ ]:
fig, axd = plt.subplot_mosaic(
    "AB;CD",
    per_subplot_kw={
        "A": {"projection": "polar"},
        ("C", "D"): {"xscale": "log"}
    },
)
identify_axes(axd)

If the layout is specified with the string short-hand, then we know the
Axes labels will be one character and can unambiguously interpret longer
strings in *per_subplot_kw* to specify a set of Axes to apply the
keywords to:



In [ ]:
fig, axd = plt.subplot_mosaic(
    "AB;CD",
    per_subplot_kw={
        "AD": {"projection": "polar"},
        "BC": {"facecolor": ".9"}
    },
)
identify_axes(axd)

If *subplot_kw* and *per_subplot_kw* are used together, then they are
merged with *per_subplot_kw* taking priority:



In [ ]:
axd = plt.figure(layout="constrained").subplot_mosaic(
    "AB;CD",
    subplot_kw={"facecolor": "xkcd:tangerine"},
    per_subplot_kw={
        "B": {"facecolor": "xkcd:water blue"},
        "D": {"projection": "polar", "facecolor": "w"},
    }
)
identify_axes(axd)

### Nested and alternative keys

Everything we can do with the string shorthand we can also do when
passing in a list (internally we convert the string shorthand to a nested
list), for example using spans, blanks, and *gridspec_kw*:

In [ ]:
axd = plt.figure(layout="constrained").subplot_mosaic(
    [
        ["main", "zoom"],
        ["main", "BLANK"],
    ],
    empty_sentinel="BLANK",
    width_ratios=[2, 1],
)
identify_axes(axd)

In addition, using the list input we can specify nested mosaics.  Any element
of the inner list can be another set of nested lists:



In [ ]:
inner = [
    ["inner A"],
    ["inner B"],
]

outer_nested_mosaic = [
    ["main", inner],
    ["bottom", "bottom"],
]
axd = plt.figure(layout="constrained").subplot_mosaic(
    outer_nested_mosaic, empty_sentinel=None
)
identify_axes(axd, fontsize=36)

We can also pass in a 2D NumPy array to do things like



In [ ]:
mosaic = np.zeros((4, 4), dtype=int)
for j in range(4):
    mosaic[j, j] = j + 1
axd = plt.figure(layout="constrained").subplot_mosaic(
    mosaic,
    empty_sentinel=0,
)
identify_axes(axd)

In [ ]:
[a for a in np.nditer(np.array([[1,2],[3,4]]))]

### Exercise: make the logo for the tutorial

<img src='../logo.png'></img>

The functions which add actual content to axes are provided.

Hint: use `identify_axes` for the axes with words (other than the logo)

Your random data will be different, so the image in the upper right and the histogram will be slightly different.

In [ ]:
from PIL import Image

def plot_image(ax):
    ax.imshow(np.random.random((20, 20)))
    ax.set(yticklabels=[], xticklabels=[])

def plot_hist(ax):
    hist_data = np.random.randn(1_500)
    ax.hist(hist_data)
    ax.set(yticklabels=[], xticklabels=[])
    

def plot_mpl_logo(ax):
    logo = Image.open("../logo2.png")
    ax.imshow(logo)
    ax.set(yticklabels=[], xticklabels=[])


### Challenge Exercise

Create a large (>12) array of axes with meaningful names

Some examples:

- A periodic table
- A stylized (square) map of the US or Europe

### Resources

- https://matplotlib.org/stable/tutorials/intermediate/arranging_axes.html
- https://matplotlib.org/stable/gallery/subplots_axes_and_figures/subplots_demo.html
- https://matplotlib.org/stable/gallery/subplots_axes_and_figures/mosaic.html